# Write a simple script that generates the main image of the company's website given the brief description of the company.


In [150]:
import tensorflow as tf
from tensorflow.keras import layers, models
from transformers import pipeline
from PIL import Image
from IPython.display import display
import os
import numpy as np

# image Generation based on the company description

def analyze_description(description):
    nlp = pipeline("ner")  
    entities = nlp(description)
    
    # Extract keywords from entities
    keywords = [entity["word"] for entity in entities if entity["label"] in ["ORG", "PRODUCT", "PERSON"]]
    # Find theme based on keywords 
    primary_theme = determine_primary_theme(keywords)
    
    return primary_theme


def generate_image(primary_theme):
    latent_dim = 100
    # Load the pre-trained VAE model
    vae = build_vae(input_shape=(28, 28, 1), latent_dim=100)
    vae.load_weights("vae_weights.h5")


    np.random.seed(hash(primary_theme) % 2**32)  
    noise = np.random.normal(0, 1, size=(1, 28, 28, 1))

    generated_image = vae.predict(noise)

    # Reshape the generated image to (28x28 for MNIST)
    generated_image = generated_image.reshape((28, 28))

    return generated_image


def display_and_save_image(generated_image):
    # Convert the floating-point  to (uint8) format
    generated_image = (generated_image * 255).astype(np.uint8)

    # Convert to grayscale
    pil_image = Image.fromarray(generated_image, mode='L')
    # Display 
#     display(pil_image)

    # Save the image to a file
    save_path = "generated_image.png"
    pil_image.save(save_path)
    print(f"Image saved as {save_path}")

# Example usage:
company_description = "A modern fashion company producing different kinds of clothing for men and women."
primary_theme = analyze_description(company_description)
generated_image = generate_image(primary_theme)
display_and_save_image(generated_image)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized 

1/1 [==============================] - 0s 264ms/step
Image saved as generated_image.png
